## Языковое моделирование

Языковое моделирование заключается в приписывании вероятности последовательности слов. Сейчас языковые модели используются практически во всех nlp задачах. Всякие Берты и Элмо - языковые модели. 

Это достаточно сложная тема, поэтому будем разбирать постепенно. Сегодня разберём самые основы. Научимся приписывать вероятность последовательности слов и попробуем генерировать текст.

Возьмем два текста: Анну Каренину и Бесов. 

In [1]:
dostoevsky = open('besy_dostoevsky.txt', encoding='cp1251').read()
tolstoy = open('anna_karenina.txt').read()

Анна Каренина немного больше.

In [2]:
print("Длина Бесов Достоевского -", len(dostoevsky))
print("Длина Анны Карениной Толстого - ", len(tolstoy))

Длина Бесов Достоевского - 1293557
Длина Анны Карениной Толстого -  1710408


Напишем простую функцию для нормализации. 

In [14]:
from string import punctuation
import numpy as np

def normalize(text):
    normalized_text = [word.strip(punctuation) for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


Сравним тексты по словам

In [15]:
norm_dostoevsky = normalize(dostoevsky)
norm_tolstoy = normalize(tolstoy)

In [16]:
print("Длина Бесов Достоевского в токенах -", len(norm_dostoevsky))
print("Длина Анны Карениной Толстого в токенах - ", len(norm_tolstoy))

Длина Бесов Достоевского в токенах - 208453
Длина Анны Карениной Толстого в токенах -  281201


Бесы короче, но уникальных слов там больше!

In [17]:
print("Уникальных лемм в Бесах -", len(set(norm_dostoevsky)))
print("Уникальный лемм в Анне Карениной - ", len(set(norm_tolstoy)))

Уникальных лемм в Бесах - 32547
Уникальный лемм в Анне Карениной -  34820


Посчитаем, сколько раз встречаются слова и выведем самые частотные.

In [18]:
from collections import Counter

In [19]:
vocab_dostoevsky = Counter(norm_dostoevsky)
vocab_tolstoy = Counter(norm_tolstoy)


In [20]:
vocab_dostoevsky.most_common(10)

[('и', 8599),
 ('—', 7227),
 ('в', 4734),
 ('не', 4707),
 ('что', 3547),
 ('я', 3377),
 ('он', 2489),
 ('с', 2419),
 ('на', 2359),
 ('но', 1833)]

In [21]:
vocab_tolstoy.most_common(10)

[('и', 12885),
 ('–', 11490),
 ('не', 6517),
 ('что', 5721),
 ('в', 5717),
 ('он', 5531),
 ('на', 3594),
 ('она', 3418),
 ('с', 3324),
 ('я', 3147)]

Сравнивать употребимость конкретных слов в разных текстах в абсолютных числах неудобно. Нормализуем счётчики на размеры текстов. Так у нас получается вероятность слова.

In [22]:
probas_dosoevsky = Counter({word:c/len(norm_dostoevsky) for word, c in vocab_dostoevsky.items()})
probas_dosoevsky.most_common(20)

[('и', 0.0412515051354502),
 ('—', 0.03466968573251524),
 ('в', 0.02271015528680326),
 ('не', 0.022580629686308185),
 ('что', 0.017015826109482712),
 ('я', 0.016200294550810016),
 ('он', 0.011940341467860861),
 ('с', 0.01160453435546622),
 ('на', 0.011316699687699385),
 ('но', 0.008793349100276801),
 ('вы', 0.008419164032179917),
 ('а', 0.008160112831189765),
 ('как', 0.00770917185168839),
 ('это', 0.006720939492355591),
 ('же', 0.0060541225120290905),
 ('его', 0.006025339045252407),
 ('так', 0.005440075220793176),
 ('к', 0.005392102776165371),
 ('всё', 0.004653327128897162),
 ('она', 0.004566976728567111)]

In [23]:
probas_tolstoy = Counter({word:c/len(norm_tolstoy) for word, c in vocab_tolstoy.items()})
probas_tolstoy.most_common(20)

[('и', 0.04582131642490603),
 ('–', 0.04086045213210479),
 ('не', 0.023175593258914443),
 ('что', 0.020344877863165495),
 ('в', 0.020330653162684342),
 ('он', 0.019669204590310845),
 ('на', 0.012780893382313719),
 ('она', 0.012155006561143097),
 ('с', 0.01182072609983606),
 ('я', 0.011191283103545151),
 ('как', 0.009395414667799902),
 ('его', 0.009121589183537754),
 ('но', 0.009057578031372577),
 ('это', 0.007848478490474785),
 ('к', 0.007044782913289782),
 ('ее', 0.006390446691156859),
 ('все', 0.005889025999196305),
 ('было', 0.005871245123594866),
 ('сказал', 0.005007094569364974),
 ('так', 0.004985757518643248)]

Эти вероятности уже можно использовать, чтобы ответить на вопрос - кто из авторов сказал бы такую фразу?

In [28]:
phrase = 'Все смешалось в доме облонских'

prob = Counter({'tolstoy':0, 'dostoevsky':0})

for word in normalize(phrase):
    prob['dostoevsky'] += np.log(probas_dosoevsky.get(word, 0.00001))
    prob['tolstoy'] += np.log(probas_tolstoy.get(word, 0.00001))



In [29]:
prob.most_common()

[('tolstoy', -39.40673502639174), ('dostoevsky', -40.89446321918831)]

Результаты получаются не очень точные. Возможно это из-за того, что мы считаем слова незовисымыми друг от друга. А это очевидно не так

По-хорошему вероятность последовательности нужно расчитывать по формуле полной вероятности. Но у нас не очень большие тексты и мы не можем получить вероятности для длинных фраз (их просто может не быть в текстах). Поэтому мы воспользуемся предположением Маркова и будем учитывать только предыдущее слово.

Чтобы расчитать вероятность с таким предположением, нам достаточно найти количество вхождений для каждого биграмма.

In [31]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

Для того, чтобы у нас получились честные вероятности и можно было посчитать вероятность первого слова, нам нужно добавить тэг маркирующий начало предложений \< start \>

Дальше мы попробуем сгенерировать текст, используя эти вероятности, и нам нужно будет когда-то остановится. Для этого добавим тэг окончания \< end \>

Ну и поделим все на предложения

In [32]:
sentences_dostoevsky = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(dostoevsky)]
sentences_tolstoy = [['<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(tolstoy)]

In [33]:
unigrams_dostoevsky = Counter()
bigrams_dostoevsky = Counter()

for sentence in sentences_dostoevsky:
    unigrams_dostoevsky.update(sentence)
    bigrams_dostoevsky.update(ngrammer(sentence))


unigrams_tolstoy = Counter()
bigrams_tolstoy = Counter()

for sentence in sentences_tolstoy:
    unigrams_tolstoy.update(sentence)
    bigrams_tolstoy.update(ngrammer(sentence))


In [34]:
len(unigrams_dostoevsky)

32474

In [35]:
bigrams_tolstoy.most_common(10)

[('<start> –', 6632),
 ('– сказал', 1049),
 ('<start> он', 999),
 ('<start> и', 768),
 ('<start> она', 698),
 ('<start> но', 649),
 ('– я', 642),
 ('что он', 638),
 ('– сказала', 626),
 ('<start> я', 531)]

Чтобы посчитать условную вероятность мы можем поделить количество вхождений на количество вхождений первого слова.

In [36]:
phrase = 'Нужно быть действительно великим человеком, чтобы суметь устоять даже против здравого смысла.'
# phrase = 'Все смешалось в доме облонских'
prob = Counter()
for ngram in ngrammer(['<start>'] + normalize(phrase) + ['<end>']):
    word1, word2 = ngram.split()
    if word1 in unigrams_dostoevsky and ngram in bigrams_dostoevsky:
        prob['dostoevsky'] += np.log(bigrams_dostoevsky[ngram]/unigrams_dostoevsky[word1])
    else:
        prob['dostoevsky'] += -10
    if word1 in unigrams_tolstoy and ngram in bigrams_tolstoy:
        prob['tolstoy'] += np.log(bigrams_tolstoy[ngram]/unigrams_tolstoy[word1])
    else:
        prob['tolstoy'] += -10



In [37]:
prob.most_common()

[('tolstoy', -21.907500409441923), ('dostoevsky', -50.33963928073034)]

Работает получше. Мы воспользовались небольшим хаком - для слов или биграммов, которых не было у нас в словаре, прибавляли низкую вероятность. Исправить это по-нормальному - сложно, придется подробнее разбираться с вероятностями, сглаживаниями и заменой неизвестных слов. Если интрересно - в книге Журафского про это есть.

Проблем с неизвестными словами у нас не будет, если мы будем пытаться сгенерировать новый текст. Давайте попробуем это сделать.

In [38]:
matrix_dostoevsky = np.zeros((len(unigrams_dostoevsky), 
                   len(unigrams_dostoevsky)))
id2word_dostoevsky = list(unigrams_dostoevsky)
word2id_dostoevsky = {word:i for i, word in enumerate(id2word_dostoevsky)}


for ngram in bigrams_dostoevsky:
    word1, word2 = ngram.split()
    matrix_dostoevsky[word2id_dostoevsky[word1]][word2id_dostoevsky[word2]] =  (bigrams_dostoevsky[ngram]/
                                                                     unigrams_dostoevsky[word1])



In [39]:
# создадим матрицу вероятностей перейти из 1 слов в другое
matrix_tolstoy = np.zeros((len(unigrams_tolstoy), 
                   len(unigrams_tolstoy)))

id2word_tolstoy = list(unigrams_tolstoy)
word2id_tolstoy = {word:i for i, word in enumerate(id2word_tolstoy)}


# вероятность расчитываем точно также
for ngram in bigrams_tolstoy:
    word1, word2 = ngram.split()
    matrix_tolstoy[word2id_tolstoy[word1]][word2id_tolstoy[word2]] =  (bigrams_tolstoy[ngram]/
                                                                     unigrams_tolstoy[word1])



Для генерации нам понадобится функция np.random.choice , которая выбирает случайный объект из заданных. Ещё в неё можно подать вероятность каждого объекта и она будет доставать по ним (не только максимальный по вероятности)

In [44]:

def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx])
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [45]:
print(generate(matrix_dostoevsky, id2word_dostoevsky, word2id_dostoevsky).replace('<end>', '\n'))

ограничусь лишь только есть каждый фунтов 
 она уже твердо проговорила варвара петровна измучившая себя 
 пошли как вы считаете поступок в голове» но он подсюсюкивает и те минуты еще не посмотрел николай всеволодович громко истину 
 вы 
 да вам просьбу подам посажу и все эти люди говорят философы начало эстетическое препровождение времени определены 
 вы отжившего века и вы… вы знаете… в книгах которые вы ее испугом 
 дело которое окружало юлию михайловну ее быстрого пронзительного взгляда вдруг поразила одна до которых уцелели теперь я застал арину прохоровну 
 в стену 
 я помню ваше пожертвование если он


In [47]:
print(generate(matrix_tolstoy, id2word_tolstoy, word2id_tolstoy).replace('<end>', '\n'))

– только кое-где в детстве еще мясистый лист бумаги исписали я кити 
 – нет таких отношениях как расчет что же я сама простила бы занятая укладываньем своих свиданиях с вод князь говорит что понемногу припоминала не нужно 
 она первая бросилась под угрозой не хочу 
 она подняла к расставленному карточному столу и не читая приписала внизу слышны были слова не ошибаешься 
 – продолжала решительно страстно хотя и он англичанину шорнику и на слово «немножко» 
 интересно 
 что будет доказано что приближается объяснение она пробудет еще этого дня но зато в петербурге и должно было еще нет


## Коллокации

Коллокации - это устойчивые выражения, состоящие из двух и более слов. Устойчивые - значит, что они часто используются вместе. Также часто значения коллокации не могут быть выведены лишь из значений, входящих в них слов.

In [51]:
import itertools
from pymorphy2 import MorphAnalyzer
from collections import Counter, defaultdict
import numpy as np
from string import punctuation
morph = MorphAnalyzer()

def normalize(text):
    normalized_text = [morph.parse(word.strip(punctuation))[0].normal_form for word \
                                                            in text.lower().split()]
    normalized_text = [word for word in normalized_text if word]
    return normalized_text


def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams
    

Предобработаем почти также, только теперь нам не нужны тэги начала и конца.

In [52]:
sentences_dostoevsky =  [normalize(text) for text in sent_tokenize(dostoevsky)]
sentences_tolstoy =  [normalize(text) for text in sent_tokenize(tolstoy)]


В списке много всяких чисел, однобуквеных слов и стоп-слов. 

Добавим какие-нибудь ограничения к коду выше, чтобы биграммы получались почище.

In [53]:
from nltk.corpus import stopwords

In [54]:
stops = set(stopwords.words('russian') + ['–'])

In [55]:
def ngrammer(tokens, stops, n=2):
    ngrams = []
    tokens = [token for token in tokens if token not in stops]
    for i in range(0,len(tokens)-n+1):
        ngrams.append('_'.join(tokens[i:i+n]))
    return ngrams

In [56]:
word_counter = Counter()

for text in sentences_tolstoy:
    word_counter.update(ngrammer(text, n=2, stops=stops))


In [57]:
word_counter.most_common(15)

[('алексей_александр', 577),
 ('степан_аркадьй', 549),
 ('сергей_иван', 294),
 ('дарья_александр', 209),
 ('весь_это', 171),
 ('сказать_левин', 155),
 ('сказать_степан', 114),
 ('лидий_иван', 104),
 ('сказать_вронский', 88),
 ('сказать_анна', 88),
 ('знать_это', 87),
 ('говорить_это', 86),
 ('агафья_михайло', 76),
 ('графиня_лидий', 74),
 ('сказать_кить', 62)]

В списке есть коллокации, которые попали в список из-за того, что одно слово очень частотное и вообще встречается много в каких контекстах. Нас скорее интересуют случаи, когда слова в большинстве случаев встречаются вместе. Для этого мы можем придумать какие-нибудь формулы, учитывающие частоты слов по отдельности и общую частоту.

Самый простой способ - взять количество упоминаний биграма и поделить на сумму количеств упоминаний слов по отдельности. 

Такая формула называется PMI.

In [58]:
def scorer_simple(word_count_a, word_count_b, bigram_count, *args):
    try:
        score = bigram_count/((word_count_a+word_count_b))
    
    except ZeroDivisionError:
        return 0
    
    return score

Сделаем функцию, которая будет делать счетчики для слов и биграммов.

In [59]:
def collect_stats(texts, stops):
    ## соберем статистики для отдельных слов
    ## и биграммов
    
    unigrams = Counter()
    bigrams = Counter()
    
    for text in texts:
        unigrams.update(text)
        bigrams.update(ngrammer(text, stops, 2))
    
    return unigrams, bigrams

И функцию, которая пройдет по всем биграммам и вычислит для них нашу метрику.

In [60]:
def score_bigrams(unigrams, bigrams, scorer, threshold=-100000, min_count=5):
    ## посчитаем метрику для каждого нграмма
    bigram2score = Counter()
    len_vocab = len(unigrams)
    for bigram in bigrams:
        score = scorer(unigrams[bigram[0]], unigrams[bigram[1]], 
                       bigrams[bigram], len_vocab, min_count)
        
        ## если метрика выше порога, добавляем в словарик
        if score > threshold:
            bigram2score[bigram] = score
    
    return bigram2score

In [61]:
unigrams, bigrams = collect_stats(sentences_tolstoy, stops)

In [62]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_simple)

Проблема с таким подходом в том, что на самом верху окажутся слова, которые встречают по одному разу.

In [63]:
bigram2score.most_common(15)

[('человек_который', 61.0),
 ('знать_это', 43.5),
 ('первое_время', 32.0),
 ('это_мочь', 30.5),
 ('это_весь', 29.0),
 ('это_дело', 28.5),
 ('это_время', 26.0),
 ('графиня_лидий', 24.666666666666668),
 ('левин_чувствовать', 24.0),
 ('несмотря_весь', 23.0),
 ('дело_который', 22.0),
 ('это_самый', 22.0),
 ('левин_видеть', 22.0),
 ('левин_мочь', 19.0),
 ('левин_понять', 19.0)]

Поэтому можно немного переделать оценивающую функцию, добавив минимальное число вхождений для биграмма.

In [64]:
def scorer(word_count_a, word_count_b, bigram_count, len_vocab, min_count):
    try:
        score = ((bigram_count - min_count) / ((word_count_a + word_count_b)))
    except ZeroDivisionError:
        return 0
    
    return score

In [65]:
bigram2score = score_bigrams(unigrams, bigrams, scorer)

In [66]:
bigram2score.most_common(15)

[('человек_который', 56.0),
 ('знать_это', 41.0),
 ('это_мочь', 28.0),
 ('первое_время', 27.0),
 ('это_весь', 26.5),
 ('это_дело', 26.0),
 ('это_время', 23.5),
 ('графиня_лидий', 23.0),
 ('это_самый', 19.5),
 ('левин_чувствовать', 19.0),
 ('несмотря_весь', 18.0),
 ('дело_который', 17.0),
 ('левин_видеть', 17.0),
 ('это_сказать', 16.0),
 ('друг_друг', 15.333333333333334)]

В статье про Word2Vec для создания нграммов использовалась такая функция:

In [67]:
def scorer_w2v(word_count_a, word_count_b, bigram_count, len_vocab, min_count=10):

    try:
        score = ((bigram_count - min_count) / (word_count_a * word_count_b)) * len_vocab
    except ZeroDivisionError:
        return 0
    
    return score

Посмотрим, отличается ли она от нашей.

In [68]:
bigram2score = score_bigrams(unigrams, bigrams, scorer_w2v)

In [69]:
bigram2score.most_common(15)

[('ребёнок_который', 34925.333333333336),
 ('решить_ехать', 14968.0),
 ('редко_бывать', 14968.0),
 ('решить_это', 9978.666666666666),
 ('решительно_знать', 9978.666666666666),
 ('ребёнок_мочь', 4989.333333333333),
 ('решительно_понимать', 4989.333333333333),
 ('mademoiselle_linon', 2494.6666666666665),
 ('железный_дорога', 2204.589147286822),
 ('женщина_который', 2204.589147286822),
 ('желать_это', 1624.4341085271317),
 ('сергей_иван', 1198.2692520775622),
 ('жена_посланник', 1160.3100775193798),
 ('брат_николай', 1116.0350877192982),
 ('женщина_это', 464.1240310077519)]

Во всех случаях выше мы считали нграммами только слова, которые встречаются друг за другом. Но в нграммы часто можно ещё что-то вставить. Например, "принять участие" может превратиться в "принять самое активное/непосредственное участие". 

Чтобы отловить такие случаи можно считать нграммами слова, которые встречаются внутри какого-то окна. И считать по ним все те же метрики.

Можно ещё посчитать стандартное отклонение расстояния между двумя словами. Если оно маленькое - слова обычно стоят на строгой позиции по отношению друг к другу.

In [70]:
from collections import defaultdict
def get_window_stats(texts, window=8):
    
    bigrams = defaultdict(list)
    
    # проходим окном по текстам 
    # берем первое слово и считаем его целевым
    # проходим по остальным словам и их индексам
    # добавляем в словарь пары (целевое слов, текущее слово)
    # и добавляем индекс текущего в список этой пары
    # так мы получаем (слово_1,слово_2):[1,2,1,1,3,2]
    # порядок в этом случае учитывается - (слово_2, слово_1) - другая запись
    for text in texts:
        for i in range(len(text)-window):
            words = list(enumerate(text[i:i+window]))
            target = words[0][1]
            for j, word in words[1:]:
                bigrams[(target, word)].append(j)
    
    bigrams_stds = Counter()
    for bigram in bigrams:
        # выкидываем биграмы встретившиеся < 5 раз
        if len(bigrams[bigram]) > 5:
            bigrams_stds[bigram] = np.std(bigrams[bigram])
    
    return bigrams_stds

In [71]:
bigram2std = get_window_stats(sentences_dostoevsky)

In [72]:
bigram2std.most_common()[:-20:-1]

[(('софья', 'матвей'), 0.0),
 (('большой', 'дорога'), 0.0),
 (('ваш', 'превосходительство'), 0.0),
 (('знаешь', 'ли'), 0.0),
 (('общий', 'дело'), 0.0),
 (('арин', 'прохор'), 0.0),
 (('вслед', 'за'), 0.0),
 (('семён', 'яков'), 0.0),
 (('воротиться', 'домой'), 0.0),
 (('из', 'сила'), 0.0),
 (('cher', 'он'), 0.0),
 (('алексей', 'егор'), 0.0),
 (('господин', 'кармазин'), 0.0),
 (('чуть', 'ли'), 0.0),
 (('артемий', 'павло'), 0.0),
 (('замечать', 'что'), 0.0),
 (('какой-то', 'особенный'), 0.0),
 (('с', 'постель'), 0.0),
 (('ради', 'бог'), 0.0)]

Можно применять расширить размер нграмма, а можно последовательно преобразовывать один и тот же текст, на каждом шагу собирая новые биграммы.

Напишием такую функцию.

In [73]:
def bigram_text(text, bigram2score):
    new_text = []
    i = 0
    
    while i < (len(text)-1):
        bigram = '_'.join((text[i], text[i+1]))
        if bigram in bigram2score:
            new_text.append(bigram)
            i += 2
        else:
            new_text.append(text[i])
            i += 1
    else:
        if i == (len(text)-1):
            new_text.append(text[i])
    
    return new_text

In [74]:
unigrams, bigrams = collect_stats(sentences_dostoevsky, stops)

In [75]:
bigram2score = score_bigrams(unigrams, bigrams, scorer)

In [76]:
sentences_dostoevsky_2 = [bigram_text(sent, bigram2score) for sent in sentences_dostoevsky]

In [77]:
unigrams, bigrams = collect_stats(sentences_dostoevsky_2, stops)

In [78]:
trigram2score = score_bigrams(unigrams, bigrams, scorer)

In [79]:
sentences_dostoevsky_3 = [bigram_text(sent, trigram2score) for sent in sentences_dostoevsky_2]

In [84]:
sentences_dostoevsky_3[4]

['домовый', 'ли', 'хоронить_ведьма_ль_замуж', 'выдавать']

По этой ссылке можно прочитать про другие метрики.

http://www.scielo.org.mx/scielo.php?script=sci_arttext&pid=S1405-55462016000300327#t1

### Все готовое

Писать все это самому конечно не обязательно.

Удобно пользоваться phraser из gensim'а. Он собирает статистику по корпусу, а затем склеивает слова в биграммы. Так как мы сделали выше. 

In [171]:
import gensim

In [172]:
# собираем статистики
ph = gensim.models.Phrases(texts)

In [173]:
# преобразовывать можно и через ph, но так быстрее 
p = gensim.models.phrases.Phraser(ph)

По умолчанию там используется метрики из статьи про ворд2век и ещё есть нормализованные pmi.
Если не нравятся функции оценки, то ему можно подать любую другую функцию. Интерфейс у функции там почти точно такой же как и у наших.

In [186]:
# собираем статистики по уже забиграммленному тексту
ph2 = gensim.models.Phrases(p[texts])
p2 = gensim.models.phrases.Phraser(ph2)

In [187]:
p2[p[texts[0]]][:20]

['многие',
 'интересоваться',
 'зачем',
 'нужный',
 '«яблоку»',
 'молодёжный',
 'фракция',
 'основной_задача',
 '«молодёжный',
 '«яблока»',
 'являться',
 'привлечение',
 'молодая_человек',
 'к',
 'участие',
 'в',
 'выборы',
 'и',
 'деятельность',
 'партия']

Ну и наконец нграммы есть в нлтк. Тут больше метрик, но преборазователь слов в нграммы нужно написать самому.

In [177]:
import nltk
from nltk.collocations import *

In [178]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
trigram_measures = nltk.collocations.TrigramAssocMeasures()

In [179]:
finder2 = BigramCollocationFinder.from_documents(texts)

In [180]:
finder3 = TrigramCollocationFinder.from_documents(texts)

In [182]:
finder2.nbest(bigram_measures.likelihood_ratio, 20)

[('один', 'из'),
 ('тот', 'что'),
 ('а', 'также'),
 ('при', 'это'),
 ('2017', 'год'),
 ('не', 'только'),
 ('точка', 'зрение'),
 ('то', 'есть'),
 ('сей', 'пора'),
 ('тот', 'число'),
 ('куб', 'метр'),
 ('владимир', 'путин'),
 ('2016', 'год'),
 ('тот', 'же'),
 ('потому', 'что'),
 ('миллиард', 'доллар'),
 ('о', 'тот'),
 ('прежде', 'всего'),
 ('кроме', 'тот'),
 ('до', 'сей')]

In [183]:
finder3.nbest(trigram_measures.pmi, 20)

[('1947–2001»', 'monterey', 'ca'),
 ('50-летие', 'rolling', 'stones'),
 ('acs', 'nano', 'letters'),
 ('areva', 'edf', 'alstom'),
 ('armored', 'multi-purpose', 'vehicles'),
 ('atr', 'ленур', 'ислям'),
 ('bad', 'can', 'it'),
 ('bourgeois', '«эпатировать', 'буржуа»'),
 ('bundesanstalt', 'fuer', 'geowissenschaften'),
 ('can', 'it', 'be'),
 ('charge', 'ion', 'battery'),
 ('citizens', '1947–2001»', 'monterey'),
 ('commitment', 'competence', 'consensus'),
 ('corriere', 'della', 'sera'),
 ('della', 'sera', 'папа-на-покой'),
 ('diyanet', 'isleri', 'turk-islam'),
 ('dux', 'recording', 'producers'),
 ('edf', 'alstom', 'schneider'),
 ('egf', 'gazprom', 'monitor'),
 ('espanola', 'чть', 'прад')]